In [ ]:
%%capture
!pip install transformers
#!wget < link for led_train_for_testing_on_fold_0.zip?dl=0, email a.askari@liacs.leidenuniv.nl for this checkpoint>
!unzip led_train_for_testing_on_fold_0.zip?dl=0
!pip install jsonlines

In [ ]:
%%capture
from transformers import LEDForConditionalGeneration, LEDTokenizer
import torch
trained_on_colie18 = "./led_train_for_testing_on_fold_0"
tokenizer_led = LEDTokenizer.from_pretrained(trained_on_colie18)
model_led = LEDForConditionalGeneration.from_pretrained(trained_on_colie18, return_dict_in_generate=True).to("cuda")

In [ ]:
import os,sys
import random
import jsonlines
import ast
import json
import time
start = time.time()
jsonl_path = "./whole_doc_w_summ_intro_w_newline_char/separately_para_w_summ_intro_newline_char.jsonl"
cnt = 0
do_no_repeat_ngram_size =2 #fix it

with open(jsonl_path) as fp:
  for line in fp:
    cnt += 1
    if cnt % 10000 == 0:
      print("{} lines parsed".format(cnt))
    line_dict = ast.literal_eval(line)
    id_ = line_dict["id"]
    contents = line_dict["contents"]
    line = contents

    input_ids = tokenizer_led(line, max_length=8192, return_tensors="pt").input_ids.to("cuda") #cuda
    tokens_len = len(input_ids[0])

    max_len_summary = int(tokens_len*10/100)
    print("max_len_summary int(tokens_len*10/100): ", int(tokens_len*10/100))

    global_attention_mask = torch.zeros_like(input_ids)
    sequences = model_led.generate(input_ids, global_attention_mask=global_attention_mask, max_length=max_len_summary, no_repeat_ngram_size=do_no_repeat_ngram_size).sequences
    summary = tokenizer_led.batch_decode(sequences, skip_special_tokens=True, clean_up_tokenization_spaces=True)
    summary_output = ' '.join(summary)
    summary_output = summary_output.replace("\n"," ")
    print("\n----\n\ngiven the whole document to the model led arxiv, summary is: \n\t", summary)
    print("tokens len, ", tokens_len, "| max len for summary: ", max_len_summary , "\n------\n\n\n")

    with open('separately_para_w_summ_intro_summaries.jsonl', 'a') as outfile:
        entry = {"id": id_, "contents":summary_output}
        json.dump(entry, outfile)
        outfile.write('\n')
    print('It took', time.time()-start, 'seconds.')

